In [ ]:
import numpy as np
import scipy.sparse as sparse
from common import set_figure

### Make a sparse matrix in COO

In [ ]:
data = [1.0, 1.0, 1.0, 3.0, 4.4]
col = [0, 2, 2, 1, 1]
row = [0, 0, 0, 1, 2]
A = sparse.coo_matrix((data, (row, col)), shape=(3,3))

print(A.todense())
print(A.data.dtype, A.data.shape)
print(A.col.dtype, A.col.shape)
print(A.row.dtype, A.row.shape)
print(A.nnz)

#### Try it in CSR

In [ ]:
print(A.format)
A = A.tocsr()
print(A.format)
print(A.data)    # AA
print(A.indices) # JA column indices
print(A.indptr)  # IA row pointer
A[0,0] = 0.0
print(A.toarray())
print(A.nnz)
A.eliminate_zeros()
print(A.nnz)

#### Try a random sparse matrix

Let's make a *random* sparse matrix

First we'll set the density so that
$$
density = \frac{nnz(A)}{n^2}
$$

In [ ]:
n = 1000
density = 5.0 / n # 5 points per row
nnz = int(n*n*density)
print(nnz)

Now make the entries:

In [ ]:
row = np.random.randint(0, n, size=nnz)
col = np.random.randint(0, n, size=nnz)
data = np.ones(nnz, dtype=float)

A = sparse.coo_matrix((data, (row, col)), shape=(n, n))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.spy(A, marker='.', markersize=2)

In [ ]:
print(A.shape, A.nnz)

But let's make it positive definite:

In [ ]:
A.data[:] = -1.0                   # -1 for off-diagonals
rowsum = -np.array(A.sum(axis=1))  # positive rowsum
A.setdiag(rowsum.ravel())

In [ ]:
u = np.random.rand(n)
v = np.random.rand(n)

Try a SpMV

In [ ]:
%timeit v = A @ u

In [ ]:
B = A.toarray()

In [ ]:
type(B)

In [ ]:
%timeit v = B @ u

#### Let's build our own:

In [ ]:
def matvec(A, u):
    """
    v = A @ u
    
    Assume CSR
    """
    v = np.zeros_like(u)
    nrows = A.shape[0]
    for i in range(nrows): # go over all rows
        for jj in range(A.indptr[i], A.indptr[i+1]):
            col = A.indices[jj]
            v[i] += A.data[jj] * u[col]
    return v

In [ ]:
A = A.tocsr()

In [ ]:
v = A @ u
v2 = matvec(A, u)

In [ ]:
np.abs(v-v2).max()

In [ ]:
%timeit v = matvec(A, u)

In [ ]:
A.data.shape

In [ ]:
A.indices.shape

In [ ]:
A.indptr.shape